# Final Project
### Import precomp libs

In [1]:
import spacy
nlp = spacy.load('en') # this should take some time like 10s to load
%load_ext line_profiler

In [2]:
#import the spacy english model
# "python -m spacy download en"   to download english models

import pickle 
import numpy as np
import scipy as sp
import pandas as pd
from collections import Counter
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import coo_matrix, hstack

In [3]:
# PROJECT structure: data directory with the two csvs of train,test and the ipython notbook in its own directory
# loading the data
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
print train.shape, test.shape

(404290, 6) (2345796, 3)


In [4]:
pd.isnull(test.loc[379205]['question2'])

True

In [5]:
from nltk.stem.porter import PorterStemmer
def stem(l):
    porter = PorterStemmer()
    for w in l:
        porter.stem(w)
stem('What are the stories of Kohinoor (Koh-i-Noor) Diamond? men walked to the stores to buy shirts')

In [6]:
def parse(sentence):
    bow = []
    sentence = sentence.decode('utf-8')
    p = nlp(sentence)
    for word in p:
        if word.is_stop: continue
        bow.append(word.lemma_)
    return Counter(bow)
        
parse('What are the stories of Kohinoor (Koh-i-Noor) Diamond? men walked to the stores to buy shirts')

Counter({u'(': 1,
         u')': 1,
         u'-': 2,
         u'?': 1,
         u'buy': 1,
         u'diamond': 1,
         u'koh': 1,
         u'kohinoor': 1,
         u'man': 1,
         u'noor': 1,
         u'shirt': 1,
         u'store': 1,
         u'story': 1,
         u'walk': 1})

In [8]:
def make_vecs(corpus, results=False):
    vecs = []
    results = []
    n = len(corpus)
    for i in range(n): 
        if i % 10000 == 0: print i
        f = corpus.loc[i]
        q1, q2 = f['question1'], f['question2']
        try: 
            p1 = parse(q1)
            p2 = parse(q2)
            diff1 = p1 - p2
            diff2 = p2 - p1
            vecs.append((diff1, diff2))
            if results: results.append(f['is_duplicate'])
        except Exception as e:
            vecs.append('')
            print(i, e)
    return vecs, results

In [ ]:
# fv_train, target = make_vecs(train, results=True)

In [ ]:
# fv_test, results = make_vecs(test)

In [9]:
# with open('cfv_train','w') as f:
#     pickle.dump(fv_train_,f)
# with open('ctarget','w') as f:
#     pickle.dump(target,f)
# # with open('fv_test','w') as f:
#     pickle.dump(fv_test,f)

NameError: name 'fv_train_' is not defined

## load from pickle

In [10]:
fv_train = pickle.load(open('cfv_train', 'rb'))
fv_test = pickle.load(open('cfv_test', 'rb'))
target = pickle.load(open('ctarget', 'rb'))

In [11]:
print(fv_train[10])

(Counter({u'slit': 1, u'separation': 1, u'biprism': 1, u'fresnel': 1, u'method': 1, u'find': 1}), Counter({u'laptops': 1, u'thing': 1, u'reliability': 1, u'durability': 1, u'technician': 1, u'component': 1, u'tell': 1}))


In [50]:
v1, v2 = zip(*fv_train_p)

In [52]:
v1 = v1[:1000]

In [55]:
def vectorize(vectors, vectorizer=None):
    for i, v in enumerate(vectors):
        if v == '': vectors[i] = (Counter(), Counter())
    v1, v2 = zip(*vectors)
    if not vectorizer:
        vectorizer = DictVectorizer().fit(v1 + v2)
    x1 = vectorizer.transform(v1)
    x2 = vectorizer.transform(v2)
    X = hstack([x1,x2])
    return X, vectorizer

In [54]:
X, vectorizer = vectorize(fv_train_p)
target = target_p

#### Validation

In [26]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
n_estimators = 10
clf = OneVsRestClassifier(BaggingClassifier(SVC(kernel='linear'), max_samples=1.0 / n_estimators, n_estimators=n_estimators, n_jobs = -1))
scores = cross_val_score(clf, X, target, cv=3)
print "Log Loss: %0.5f" % scores.mean()

In [ ]:
n_estimators = 20
clfs = [RandomForestClassifier(min_samples_leaf=20, min_samples_split=10, n_jobs=-1), MultinomialNB(), SGDClassifier(loss='log'), LogisticRegression()]
# clfs = [SVC(kernel='linear', probability=True)] 

for clf in clfs:
    print clf
    scores = cross_val_score(clf, X, target, cv=3, scoring='neg_log_loss')
    print "Log Loss: %0.5f" % scores.mean()

### Testing

In [17]:
f = open('submission', 'w')
f.write('test_id,is_duplicate')

In [20]:
print fv_test_p[1]
print fv_test_p[1]

(Counter({u'24': 1, u'age': 1, u'?': 1}), Counter({u'require': 1}))
(Counter({u'24': 1, u'age': 1, u'?': 1}), Counter({u'require': 1}))


In [56]:
fv_test = vectorize(fv_test_p, vectorizer)

In [57]:
clf = LogisticRegression().fit(X, target)

In [58]:
prob = clf.predict_proba(fv_test)

/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


TypeError: float() argument must be a string or a number